## EXAMEN DE RECUPERACION DE LA INFORMACION

### SEGUNDO BIMESTRE

#### Jorge Rojas

#### CARGA Y EDIT DEL CORPUS

In [1]:
%pip install kagglehub

   ---------------------------------------- 0.0/68.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/68.0 kB ? eta -:--:--
   ------ --------------------------------- 10.2/68.0 kB ? eta -:--:--
   ---------------------------------------- 68.0/68.0 kB 916.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import kagglehub

# Descargamos el dataset
path = kagglehub.dataset_download("Cornell-University/arxiv")
print("Path to dataset files:", path)


c:\Users\gboy2\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 1.47G/1.47G [09:52<00:00, 2.67MB/s] 

Extracting files...


Path to dataset files: C:\Users\gboy2\.cache\kagglehub\datasets\Cornell-University\arxiv\versions\244


In [3]:
import json
import os
import random

# Cargamos el JSON
file_path = os.path.join(path, "arxiv-metadata-oai-snapshot.json")  

# Leer JSON 
documents = []
with open(file_path, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        try:
            doc = json.loads(line)
            documents.append(doc)
        except:
            continue

print(f"Total documentos: {len(documents)}")

# Tomar solo el 1%
subset_size = int(len(documents) * 0.01)
subset = random.sample(documents, subset_size)

print(f"Corpus reducido a {len(subset)} documentos")


Total documentos: 2792339
Corpus reducido a 27923 documentos


In [4]:
with open("arxiv_subset.json", "w", encoding="utf-8") as out:
    json.dump(subset, out, indent=2)


#### PREPROCESAMIENTO

In [5]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = text.lower()  # minúsculas
    text = re.sub(r'[^\w\s]', '', text)  # eliminar puntuación
    tokens = word_tokenize(text)  # tokenización
    tokens = [word for word in tokens if word not in stop_words]  # quitar stopwords
    return " ".join(tokens)


[nltk_data] Downloading package punkt to C:\Users\gboy2/nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gboy2/nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\gboy2/nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Error with downloaded zip file


False

In [11]:
import nltk
nltk.data.path.append("C:/Users/gboy2/nltk_data")


In [13]:
import nltk
import shutil
import os

# Borra manualmente si no lo hiciste antes
shutil.rmtree("C:/Users/gboy2/nltk_data/tokenizers/punkt", ignore_errors=True)

# Fuerza nueva descarga
nltk.download('punkt')


[nltk_data] Downloading package punkt to C:\Users\gboy2/nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\gboy2/nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [3]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

def preprocess_fallback(text):
    text = re.sub(r'[^\w\s]', '', text.lower())
    tokens = text.split()
    return " ".join([t for t in tokens if t not in ENGLISH_STOP_WORDS])


In [5]:
import json

with open("arxiv_subset.json", "r", encoding="utf-8") as f:
    subset = json.load(f)

print(f"✅ Corpus cargado con {len(subset)} documentos")


✅ Corpus cargado con 27923 documentos


In [7]:
import re

processed_docs = []
for doc in subset:
    combined_text = doc["title"] + " " + doc["abstract"]
    processed = preprocess_fallback(combined_text)  # usamos la versión sin NLTK
    processed_docs.append(processed)


In [9]:
processed_docs = []
for doc in subset:
    combined_text = doc["title"] + " " + doc["abstract"]
    processed = preprocess_fallback(combined_text)
    processed_docs.append(processed)


In [11]:
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

def preprocess_fallback(text):
    text = re.sub(r'[^\w\s]', '', text.lower())  # quitar puntuación y pasar a minúsculas
    tokens = text.split()
    return " ".join([t for t in tokens if t not in ENGLISH_STOP_WORDS])  # eliminar stopwords

# Inicializar listas
processed_docs = []
doc_ids = []
titles = []
abstracts = []

# Procesar documentos
for doc in subset:
    if "title" in doc and "abstract" in doc:
        combined = f"{doc['title']} {doc['abstract']}"
        cleaned = preprocess_fallback(combined)
        processed_docs.append(cleaned)
        doc_ids.append(doc["id"])
        titles.append(doc["title"])
        abstracts.append(doc["abstract"])


#### INDEXACION TF-IDF

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Crear el vectorizador
tfidf_vectorizer = TfidfVectorizer()

# Aplicar sobre los textos procesados
tfidf_matrix = tfidf_vectorizer.fit_transform(processed_docs)

print(f"✅ TF-IDF generado: {tfidf_matrix.shape[0]} documentos, {tfidf_matrix.shape[1]} términos únicos")


✅ TF-IDF generado: 27923 documentos, 123310 términos únicos


In [13]:
def search_tfidf(query, top_k=10):
    query_cleaned = preprocess_text(query)
    query_vec = tfidf_vectorizer.transform([query_cleaned])
    scores = (tfidf_matrix @ query_vec.T).toarray().flatten()
    top_indices = scores.argsort()[::-1][:top_k]
    
    results = []
    for idx in top_indices:
        results.append({
            "id": doc_ids[idx],
            "title": titles[idx],
            "score": scores[idx],
            "abstract_snippet": abstracts[idx][:300] + "..."
        })
    return results


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Crear vectorizador y ajustar
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(processed_docs)


In [15]:
def search_tfidf(query, top_k=10):
    query_clean = preprocess_fallback(query)  # usamos el mismo limpiador
    query_vec = tfidf_vectorizer.transform([query_clean])
    scores = (tfidf_matrix @ query_vec.T).toarray().flatten()
    top_indices = scores.argsort()[::-1][:top_k]
    
    results = []
    for idx in top_indices:
        results.append({
            "id": doc_ids[idx],
            "title": titles[idx],
            "score": round(scores[idx], 4),
            "abstract_snippet": abstracts[idx][:300] + "..."
        })
    return results


##### TEST

In [16]:
query = "quantum chromodynamics"
results = search_tfidf(query)

for r in results:
    print(f"🔹 {r['title']} (Score: {r['score']})")
    print(f"📝 {r['abstract_snippet']}\n")


🔹 Lecture notes on "Quantum chromodynamics and statistical physics" (Score: 0.5282)
📝   The concepts and methods used for the study of disordered systems have proven
useful in the analysis of the evolution equations of quantum chromodynamics in
the high-energy regime: Indeed, parton branching in the semi-classical
approximation relevant at high energies is a peculiar branching-diffus...

🔹 Effective Fields in Dense Quantum Chromodynamics (Score: 0.4744)
📝   In the high density, low temperature limit, Quantum Chromodynamics exhibits a
transition to phases characterized by color superconductivity and energy gaps
in the fermion spectra. We review some fundamental results obtained in this
area and in particular we describe the low energy effective lagran...

🔹 The universal quantum driving force to speed up a quantum computation --
  The unitary quantum dynamics (Score: 0.2417)
📝   It is shown in the paper that the unitary quantum dynamics in quantum
mechanics is the universal quantum driv

In [17]:
%pip install rank_bm25

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
def simple_tokenize(text):
    text = re.sub(r'[^\w\s]', '', text.lower())
    return [t for t in text.split() if t not in ENGLISH_STOP_WORDS]

tokenized_docs = [simple_tokenize(doc) for doc in processed_docs]


In [52]:
from rank_bm25 import BM25Okapi

bm25_model = BM25Okapi(tokenized_docs)

In [53]:
def search_bm25(query, top_k=10):
    query_tokens = simple_tokenize(query)
    scores = bm25_model.get_scores(query_tokens)
    top_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]

    results = []
    for idx in top_indices:
        results.append({
            "id": doc_ids[idx],
            "title": titles[idx],
            "score": round(scores[idx], 4),
            "abstract_snippet": abstracts[idx][:300] + "..."
        })
    return results

##### TEST

In [54]:
query = "quantum chromodynamics"
results = search_bm25(query)

for r in results:
    print(f"🔹 {r['title']} (Score: {r['score']:.2f})")
    print(f"📝 {r['abstract_snippet']}\n")


🔹 Lecture notes on "Quantum chromodynamics and statistical physics" (Score: 16.86)
📝   The concepts and methods used for the study of disordered systems have proven
useful in the analysis of the evolution equations of quantum chromodynamics in
the high-energy regime: Indeed, parton branching in the semi-classical
approximation relevant at high energies is a peculiar branching-diffus...

🔹 Effective Fields in Dense Quantum Chromodynamics (Score: 15.85)
📝   In the high density, low temperature limit, Quantum Chromodynamics exhibits a
transition to phases characterized by color superconductivity and energy gaps
in the fermion spectra. We review some fundamental results obtained in this
area and in particular we describe the low energy effective lagran...

🔹 Inhomogeneous color superconductivity and the cooling of compact stars (Score: 13.72)
📝   In this talk I discuss the inhomogeneous (LOFF) color superconductive phases
of Quantum Chromodynamics (QCD). In particular, I show the effect of

In [23]:
%pip install sentence-transformers faiss-cpu

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Cargar modelo
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Generar embeddings del corpus
corpus_embeddings = embedding_model.encode(processed_docs, show_progress_bar=True)

# Crear índice FAISS (distancia L2)
dimension = corpus_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(corpus_embeddings))

print("✅ FAISS cargado con", index.ntotal, "documentos")

c:\Users\gboy2\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Batches: 100%|██████████| 873/873 [06:13<00:00,  2.34it/s]

✅ FAISS cargado con 27923 documentos


In [25]:
def search_faiss(query, top_k=10):
    query_vec = embedding_model.encode([query])
    D, I = index.search(np.array(query_vec), top_k)

    results = []
    for idx in I[0]:
        results.append({
            "id": doc_ids[idx],
            "title": titles[idx],
            "score": round(D[0][list(I[0]).index(idx)], 4),
            "abstract_snippet": abstracts[idx][:300] + "..."
        })
    return results


##### TEST

In [26]:
results = search_faiss("quantum chromodynamics")
for r in results:
    print(f"🔹 {r['title']} (Distancia: {r['score']})")
    print(f"📝 {r['abstract_snippet']}\n")


🔹 Lecture notes on "Quantum chromodynamics and statistical physics" (Distancia: 0.7078999876976013)
📝   The concepts and methods used for the study of disordered systems have proven
useful in the analysis of the evolution equations of quantum chromodynamics in
the high-energy regime: Indeed, parton branching in the semi-classical
approximation relevant at high energies is a peculiar branching-diffus...

🔹 Quantum Chromodynamics (CERN-2014-001) (Distancia: 0.9210000038146973)
📝   I review the basics of perturbative QCD, including infrared divergences and
safety, collinear and $k_T$ factorization theorems, and various evolution
equations and resummation techniques for single- and double-logarithmic
corrections. I then elaborate its applications to studies of jet substructur...

🔹 Effective Fields in Dense Quantum Chromodynamics (Distancia: 0.9318000078201294)
📝   In the high density, low temperature limit, Quantum Chromodynamics exhibits a
transition to phases characterized by color supe

#### EVALUACION ENTRE MODELOS

In [30]:
import pandas as pd

# Ejecutar búsqueda con una misma query
query = "quantum chromodynamics"
tfidf_results = search_tfidf(query, top_k=10)
bm25_results = search_bm25(query, top_k=10)
faiss_results = search_faiss(query, top_k=10)

# Extraer IDs
tfidf_ids = [doc["id"] for doc in tfidf_results]
bm25_ids = [doc["id"] for doc in bm25_results]
faiss_ids = [doc["id"] for doc in faiss_results]

# Comparativa por ranking
df = pd.DataFrame({
    "Rank": range(1, 11),
    "TF-IDF_ID": tfidf_ids,
    "BM25_ID": bm25_ids,
    "FAISS_ID": faiss_ids
})

# Coincidencias por posición
df["TFIDF == BM25"] = df["TF-IDF_ID"] == df["BM25_ID"]
df["TFIDF == FAISS"] = df["TF-IDF_ID"] == df["FAISS_ID"]
df["BM25 == FAISS"] = df["BM25_ID"] == df["FAISS_ID"]

# Coincidencias globales 
intersection_tf_bm25 = len(set(tfidf_ids) & set(bm25_ids))
intersection_tf_faiss = len(set(tfidf_ids) & set(faiss_ids))
intersection_bm25_faiss = len(set(bm25_ids) & set(faiss_ids))

print("🔎 Coincidencias en Top-10:")
print(f"TF-IDF & BM25: {intersection_tf_bm25}")
print(f"TF-IDF & FAISS: {intersection_tf_faiss}")
print(f"BM25 & FAISS: {intersection_bm25_faiss}")

df["TFIDF_Title"] = [doc["title"] for doc in tfidf_results] # Modelo base del documento en TF-IDFF


df


🔎 Coincidencias en Top-10:
TF-IDF & BM25: 6
TF-IDF & FAISS: 3
BM25 & FAISS: 3


,Rank,TF-IDF_ID,BM25_ID,FAISS_ID,TFIDF == BM25,TFIDF == FAISS,BM25 == FAISS,TFIDF_Title
0,1,1410.6478,1410.6478,1410.6478,True,True,True,"Lecture notes on ""Quantum chromodynamics and s..."
1,2,hep-ph/0206065,hep-ph/0206065,1406.7689,True,False,False,Effective Fields in Dense Quantum Chromodynamics
2,3,1105.3573,0704.1724,hep-ph/0206065,False,False,False,The universal quantum driving force to speed u...
3,4,0704.1724,1406.7689,hep-th/9610216,False,False,False,Inhomogeneous color superconductivity and the ...
4,5,2303.03311,nucl-th/0306021,1112.6141,False,False,False,Ising Meson Spectroscopy on a Noisy Digital Qu...
5,6,1406.7689,hep-ph/9702399,quant-ph/0210119,False,False,False,Quantum Chromodynamics (CERN-2014-001)
6,7,quant-ph/0208078,hep-ex/0605089,gr-qc/0509089,False,False,False,Quantum information science as an approach to ...
7,8,0708.0261,1411.2554,1212.2055,False,False,False,An Introduction to Quantum Computing
8,9,nucl-th/0306021,2303.03311,quant-ph/0303181,False,False,False,Baryons as Relativistic Three-Quark Systems
9,10,cs/0403003,quant-ph/0209165,1611.00205,False,False,False,Genetic Algorithms and Quantum Computation


#### RAG

In [33]:
%pip install openai python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
from dotenv import load_dotenv
import os

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Verifica (opcional):
print("API Key cargada:", "Sí" if openai.api_key else "No")


API Key cargada: Sí


In [42]:
%pip install --upgrade openai

   ---------------------------------------- 0.0/764.4 kB ? eta -:--:--
   -- ------------------------------------- 41.0/764.4 kB 2.0 MB/s eta 0:00:01
   --------- ------------------------------ 174.1/764.4 kB 2.6 MB/s eta 0:00:01
   -------------------------- ------------- 501.8/764.4 kB 5.3 MB/s eta 0:00:01
   ---------------------------------------- 764.4/764.4 kB 6.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.93.0
    Uninstalling openai-1.93.0:
      Successfully uninstalled openai-1.93.0
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [43]:
from openai import OpenAI
from dotenv import load_dotenv
import os

# Cargar clave desde .env
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def generate_rag_response_openai(query, top_k=3):
    top_docs = search_faiss(query, top_k=top_k)
    contexto = "\n\n".join([f"{i+1}. {doc['abstract_snippet']}" for i, doc in enumerate(top_docs)])

    prompt = (
        f"Pregunta del usuario:\n{query}\n\n"
        f"Contexto recuperado desde documentos científicos relevantes:\n{contexto}\n\n"
        f"Con base en el contexto anterior, responde de forma clara, concisa y académica:"
    )

    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.7
    )

    return response.choices[0].message.content


In [44]:
print(generate_rag_response_openai("quantum chromodynamics"))

La cromodinámica cuántica es la teoría cuántica de las interacciones fuertes entre partículas subatómicas, que se basa en la interacción entre quarks y gluones. Es una teoría fundamental en la física de partículas que describe cómo los quarks y los gluones interactúan a través de la fuerza nuclear fuerte. La cromodinámica cuántica ha sido estudiada en diversos regímenes de energía, incluyendo el régimen de alta energía donde se utilizan métodos semi-clásicos y la transición a fases como la superconductividad de color en condiciones de alta densidad y baja temperatura. Además, se han desarrollado técnicas de resumen y evolución de ecuaciones para estudiar correcciones logarítmicas en la teoría.


##### EVALUACION Y TEST SI LA RESPUESTA USA LA INFORMACION DEL CONTEXTO

In [47]:
from collections import Counter

def evaluar_rag_response(query, top_k=3):
    # 1. Obtener documentos
    top_docs = search_faiss(query, top_k=top_k)
    context = " ".join([doc["abstract_snippet"] for doc in top_docs])

    # 2. Generar respuesta
    respuesta = generate_rag_response_openai(query, top_k=top_k)

    # 3. Palabras clave del contexto
    context_tokens = [t for t in re.sub(r'[^\w\s]', '', context.lower()).split() if t not in ENGLISH_STOP_WORDS]
    respuesta_tokens = [t for t in re.sub(r'[^\w\s]', '', respuesta.lower()).split()]

    # 4. Comparar y contar coincidencias
    context_counter = Counter(context_tokens)
    respuesta_counter = Counter(respuesta_tokens)
    coincidencias = sum((respuesta_counter & context_counter).values())

    print("Consulta:", query)
    print("Palabras clave del contexto:", len(set(context_tokens)))
    print("Palabras en respuesta:", len(respuesta_tokens))
    print("Palabras del contexto usadas en la respuesta:", coincidencias)
    print("\n Respuesta generada:")
    print(respuesta)

    return {
        "coincidencias": coincidencias,
        "longitud_contexto": len(set(context_tokens)),
        "longitud_respuesta": len(respuesta_tokens),
        "respuesta": respuesta
    }


In [48]:
evaluar_rag_response("quantum chromodynamics")

Consulta: quantum chromodynamics
Palabras clave del contexto: 68
Palabras en respuesta: 160
Palabras del contexto usadas en la respuesta: 40

 Respuesta generada:
Quantum chromodynamics (QCD) is the theory that describes the strong nuclear force, which is responsible for binding quarks together to form protons, neutrons, and other particles. It is a fundamental theory in particle physics and plays a crucial role in understanding the behavior of matter at high energies and densities. Recent research has applied concepts and methods from the study of disordered systems to analyze the evolution equations of QCD in the high-energy regime, particularly in relation to parton branching. Additionally, perturbative QCD, which deals with the interactions of quarks and gluons at high energies, has been studied extensively, including the resolution of infrared divergences and the development of factorization theorems and resummation techniques for corrections in scattering processes. In the high-d

{'coincidencias': 40,
 'longitud_contexto': 68,
 'longitud_respuesta': 160,
 'respuesta': 'Quantum chromodynamics (QCD) is the theory that describes the strong nuclear force, which is responsible for binding quarks together to form protons, neutrons, and other particles. It is a fundamental theory in particle physics and plays a crucial role in understanding the behavior of matter at high energies and densities. Recent research has applied concepts and methods from the study of disordered systems to analyze the evolution equations of QCD in the high-energy regime, particularly in relation to parton branching. Additionally, perturbative QCD, which deals with the interactions of quarks and gluons at high energies, has been studied extensively, including the resolution of infrared divergences and the development of factorization theorems and resummation techniques for corrections in scattering processes. In the high-density, low-temperature limit, QCD exhibits a phase transition to states

#### CONSULTAS queries

In [61]:
with open("queries.txt", "r", encoding="utf-8") as f:
    queries = [line.strip() for line in f if line.strip()]

# Función para extraer solo los IDs del top-k
def extract_ids(results):
    return [r["id"] for r in results]

# Procesar cada consulta
for query in queries:
    print(f" Consulta: {query}")
    print("-" * 100)

    tfidf_results = search_tfidf(query)
    bm25_results = search_bm25(query)
    faiss_results = search_faiss(query)
    rag_response = generate_rag_response_openai(query)

    # Mostrar top 10 resultados
    for name, results in zip(["TF-IDF", "BM25", "FAISS"], [tfidf_results, bm25_results, faiss_results]):
        print(f"\n🔹 {name} - Top 10 documentos:")
        for i, r in enumerate(results, 1):
            print(f"{i}. {r['id']} - {r['title'][:60]}...")

    # Extraer IDs
    tfidf_ids = extract_ids(tfidf_results)
    bm25_ids = extract_ids(bm25_results)
    faiss_ids = extract_ids(faiss_results)

    # Comparaciones
    tfidf_vs_bm25 = list(set(tfidf_ids) & set(bm25_ids))
    tfidf_vs_faiss = list(set(tfidf_ids) & set(faiss_ids))
    bm25_vs_faiss = list(set(bm25_ids) & set(faiss_ids))

    # Mostrar comparaciones
    print("\n Comparación de coincidencias en el top 10:")
    print(f"TF-IDF vs BM25: {len(tfidf_vs_bm25)} en común → {tfidf_vs_bm25}")
    print(f"TF-IDF vs FAISS: {len(tfidf_vs_faiss)} en común → {tfidf_vs_faiss}")
    print(f"BM25 vs FAISS: {len(bm25_vs_faiss)} en común → {bm25_vs_faiss}")

    # Mostrar respuesta RAG
    print("\n Respuesta generada por RAG (CHAT GTP):")
    print(rag_response)

    print("\n" + "=" * 120 + "\n")


 Consulta: diphoton production cross sections
----------------------------------------------------------------------------------------------------

🔹 TF-IDF - Top 10 documentos:
1. 2204.12853 - EXFOR-based simultaneous evaluation of neutron-induced urani...
2. 2108.07568 - Relativistic quantum-mechanical versus classical magnetic re...
3. hep-ph/0105041 - Sudakov Resummation and Finite Order Expansions of Heavy Qua...
4. 1606.08265 - Charmonia production in p+p collisions under NRQCD formalism...
5. hep-ph/9807531 - Nuclei cross sections in Extensive Air Showers...
6. 2306.12591 - Neutrino deep-inelastic scattering cross sections from 100 G...
7. hep-ex/0112021 - A Combination of Preliminary Electroweak Measurements and Co...
8. hep-ph/0111099 - More remarks on suppression of large black hole production i...
9. 0705.1051 - Quasi-elastic neutrino charged-current scattering cross sect...
10. 1409.2601 - Quarkonia production and polarization at the hadron collider...

🔹 BM25 - Top 10 docu